In [6]:
from kafka import KafkaProducer
import csv
import json
from kafka.errors import KafkaError

def read_csv(file_path):
    with open(file_path, mode='r') as file:
        csv_reader = csv.DictReader(file)
        for row in csv_reader:
            yield row

def on_send_success(record_metadata):
    print(f'Message successfully sent to topic {record_metadata.topic} partition {record_metadata.partition} offset {record_metadata.offset}')

def on_send_error(excp):
    print(f'I am an error callback: {excp}')
    # Handle the exception

def main():
    producer = KafkaProducer(
        bootstrap_servers='localhost:9092',
        value_serializer=lambda v: json.dumps(v).encode('utf-8')
    )
    
    for data in read_csv('sample_data.csv'):
        producer.send('my_topic', value=data).add_callback(on_send_success).add_errback(on_send_error)
        
    producer.flush()
    producer.close()

if __name__ == '__main__':
    main()


Message successfully sent to topic my_topic partition 0 offset 500
Message successfully sent to topic my_topic partition 0 offset 501
Message successfully sent to topic my_topic partition 0 offset 502
Message successfully sent to topic my_topic partition 0 offset 503
Message successfully sent to topic my_topic partition 0 offset 504
Message successfully sent to topic my_topic partition 0 offset 505
Message successfully sent to topic my_topic partition 0 offset 506
Message successfully sent to topic my_topic partition 0 offset 507
Message successfully sent to topic my_topic partition 0 offset 508
Message successfully sent to topic my_topic partition 0 offset 509
Message successfully sent to topic my_topic partition 0 offset 510
Message successfully sent to topic my_topic partition 0 offset 511
Message successfully sent to topic my_topic partition 0 offset 512
Message successfully sent to topic my_topic partition 0 offset 513
Message successfully sent to topic my_topic partition 0 offset

In [9]:
from kafka import KafkaConsumer
import json
import mysql.connector
from mysql.connector import Error

def insert_into_db(data):
    try:
        conn = mysql.connector.connect(
            host='gateway01.ap-southeast-1.prod.aws.tidbcloud.com',
            port=4000,
            user="2tQpfsQzrrg6wDM.root",
            password="IVvTfpI0f4CJtHBB",
            database="kafka",
        )
        cursor = conn.cursor()
        
        add_data = ("INSERT INTO your_table "
                    "(id, name, age, city) "
                    "VALUES (%s, %s, %s, %s)")
        
        data_tuple = (data['id'], data['name'], data['age'], data['city'])
        cursor.execute(add_data, data_tuple)
        
        conn.commit()
        print(f"Inserted data: {data}")
    except Error as e:
        print(f"Error: {e}")
    finally:
        if cursor:
            cursor.close()
        if conn:
            conn.close()

def main():
    print("Starting Kafka consumer...")
    consumer = KafkaConsumer(
        'my_topic',
        bootstrap_servers='localhost:9092',
        auto_offset_reset='earliest',
        enable_auto_commit=True,
        group_id='my_group',
        value_deserializer=lambda m: json.loads(m.decode('utf-8')),
        consumer_timeout_ms=10000  # Timeout after 10 seconds if no message is received
    )
    
    for message in consumer:
        data = message.value
        print(f"Received data: {data}")
        insert_into_db(data)
    
    print("No more messages in the topic or consumer timed out.")
    consumer.close()

if __name__ == '__main__':
    main()


Starting Kafka consumer...
Received data: {'id': '1', 'name': 'Person 1', 'age': '21', 'city': 'City 1'}
Inserted data: {'id': '1', 'name': 'Person 1', 'age': '21', 'city': 'City 1'}
Received data: {'id': '2', 'name': 'Person 2', 'age': '22', 'city': 'City 2'}
Inserted data: {'id': '2', 'name': 'Person 2', 'age': '22', 'city': 'City 2'}
Received data: {'id': '3', 'name': 'Person 3', 'age': '23', 'city': 'City 3'}
Inserted data: {'id': '3', 'name': 'Person 3', 'age': '23', 'city': 'City 3'}
Received data: {'id': '4', 'name': 'Person 4', 'age': '24', 'city': 'City 4'}
Inserted data: {'id': '4', 'name': 'Person 4', 'age': '24', 'city': 'City 4'}
Received data: {'id': '5', 'name': 'Person 5', 'age': '25', 'city': 'City 5'}
Inserted data: {'id': '5', 'name': 'Person 5', 'age': '25', 'city': 'City 5'}
Received data: {'id': '6', 'name': 'Person 6', 'age': '26', 'city': 'City 6'}
Inserted data: {'id': '6', 'name': 'Person 6', 'age': '26', 'city': 'City 6'}
Received data: {'id': '7', 'name': 'P

KeyboardInterrupt: 